In [18]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [20]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [45]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

### Задача № 1*
поменять код функции calc_logloss таким образом чтобы ошибка вычислялась при точном предсказании

In [68]:
def calc_logloss(y, y_pred):
    y_pred_float = y_pred.copy().astype(np.float64)
    y_pred_float[y_pred_float == 0.0] = 0.1
    y_pred_float[y_pred_float == 1.0] = 0.9
    err = - np.mean(y * np.log(y_pred_float) + (1.0 - y) * np.log(1.0 - y_pred_float))
    return err

изменим код функции таким образом, чтобы значения 0 и 1 заменялись на 0.1 и на 0.9 соотвественно.

In [73]:
#проверка функции
y1 = np.array([1, 0])
y_pred1 = np.array([0.6, 0.1])
calc_logloss(y1, y_pred1)

0.30809306971190853

In [74]:
# еще одна проверка функции
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0])
calc_logloss(y1, y_pred1)

0.10536051565782628

### Задача №2 
подобрать аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [28]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [97]:
W = eval_model(X_st, y, iterations=10000, eta=1)

0 [ 0.11732727 -1.58914029  0.62030812  0.95688359] 1.1785958344356262
1000 [-10.79956087  -1.38842301  -2.43773192   9.1781253 ] 0.2316594753901533
2000 [-15.46737926  -1.78489608  -3.84058484  12.94996634] 0.19317640063439964
3000 [-19.06453024  -2.10307143  -4.89402886  15.87355904] 0.1703820922419324
4000 [-22.02639821  -2.36864314  -5.74972113  18.28151701] 0.15496509524744975
5000 [-24.55062127  -2.59641317  -6.47260633  20.33229852] 0.1437907257559476
6000 [-26.75533033  -2.7959032   -7.1002527   22.12170973] 0.13528147157282966
7000 [-28.71761855  -2.97355287  -7.65663776  23.71251207] 0.12855136478278142
8000 [-30.49042692  -3.13388494  -8.15795259  25.14787759] 0.12306625393611918
9000 [-32.11152175  -3.28017801  -8.61559398  26.45863223] 0.11848606975828083


### Задача №3
Создать функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [119]:
W

array([-33.60724757,  -3.41474448,  -9.03744936,  27.66630585])

In [120]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [121]:
def calc_pred_proba(w, X):

    A = np.squeeze(sigmoid(np.dot(X, w))) 

    return A

In [122]:
# Предскажем вероятность принадлежность к классу
p = predict(W,X_st)
p

array([3.76907782e-01, 1.43398442e-02, 9.99999995e-01, 1.09706210e-07,
       9.32650715e-01, 6.42051408e-02, 1.00000000e+00, 2.06413788e-02,
       6.05682604e-01, 9.99999459e-01])

### Задача №4
Создать функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [142]:
# lim порок отнесения предсказания к классу

def calc_pred(w, X, lim):
    m = X.shape[0]
    y_pred = np.zeros(m)
    
    A = np.squeeze(sigmoid(np.dot(X, w)))
    
    # если вероятность больше 0.5 относим к 1 классу, если меньше или равно то к 0
    for i in range(A.shape[0]):
        if (A[i] > lim): 
            y_pred[i] = 1
        elif (A[i] <= lim):
            y_pred[i] = 0
    return y_pred
    

In [145]:
# сделам предсказание класса на текущих данных
c = calc_pred(W, X_st, 0.8)
c

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

### Задача № 5*
Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [146]:
# функция для посчета accuracy
def calc_accuracy(y_pred, y):
    accuracy = np.mean(y_pred == y) * 100.0
    return accuracy

In [148]:
a = calc_accuracy(c, y)
print(f"предсказанные классы: {c} при значении веротности 0.8\n заданные значения классов:\
      {y}\n рассчитанная точность модели (accuracy): {a}")

предсказанные классы: [0. 0. 1. 0. 1. 0. 1. 0. 0. 1.] при значении веротности 0.8
 заданные значения классов:      [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
 рассчитанная точность модели (accuracy): 90.0


In [161]:
# функция создания матирцы ошибок
def confusion_matrix(y_pred, y):
    
  
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    
    for i in range(len(y)):
        if y_pred[i] == y[i] == 1:
            TP += 1 
        if y_pred[i] == 1 and y[i] != y_pred[i]:
            FP += 1    
        if y_pred[i] == 0 and y[i] != y_pred[i]:
            FN += 1
        if y_pred[i] == y[i] == 0:
            TN += 1
    
    return [TP, FP, FN, TN]


In [165]:
print(confusion_matrix(c, y))

[4, 0, 1, 5]


In [166]:
# функция расчета precision
def calc_precission(y_pred, y):
    cm = confusion_matrix(c, y)
    precission = cm[0] / (cm[0] + cm[1])
    
    return precission

In [167]:
# функция расчета recall
def calc_recall(y_pred, y):
    cm = confusion_matrix(c, y)
    recall = cm[0] / (cm[0] + cm[2])
    
    return recall

In [168]:
prec, rec = calc_precission(c, y), calc_recall(c, y)
print(prec, rec)

1.0 0.8


In [169]:
# функция расчета F1 score
def f_score(prec, rec):
    f_score = 2 * prec * rec / (prec + rec)
    return f_score

In [170]:
f1 = f_score(prec, rec)
f1

0.888888888888889

### Задача №6
Могла ли модель переобучиться? Почему?

Переобучение - это когда модель показывает очень хорошие результаты ошибки на тренировочной выборке,
и плохие на реальной выборке. Часто переобучение происходит из-за малого количества данных в обучающей выборке.
Тогда модель может "не увидеть" всех возможных вариантов как итог мы будем иметь очень плохой результат по
предсказанию в реальных данных.
Еще переобучение происходит в результате однообразных данных, когда очень много одинаковых, мало различающихся 
между собой признаков.
Если отнести сказанное к нашим данным, то да, переобучение модели возможно из-за малого количества данных в 
обучающей выборке.